In [1]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

import keras

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
cmap = plt.get_cmap('inferno')
#print
plt.figure(figsize=(10,10))

#looping and creating path for every genres 
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'./MIR/genres/{g}'):
        songname = f'./MIR/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

<Figure size 720x720 with 0 Axes>

In [3]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [4]:
file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
    
    
    #mapping
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'./MIR/genres/{g}'):    #folder names are being genereated.....still in folder
        songname = f'./MIR/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rmse(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)        
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [5]:
data = pd.read_csv('data.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00000.wav,0.349943,0.130225,1784.420446,2002.650192,3806.485316,0.083066,-113.596742,121.557302,-19.158825,...,8.810668,-3.667367,5.751690,-5.162761,0.750947,-1.691937,-0.409954,-2.300208,1.219928,blues
1,blues.00001.wav,0.340983,0.095918,1529.835316,2038.617579,3548.820207,0.056044,-207.556796,124.006717,8.930562,...,5.376802,-2.239119,4.216963,-6.012273,0.936109,-0.716537,0.293875,-0.287431,0.531573,blues
2,blues.00002.wav,0.363603,0.175573,1552.481958,1747.165985,3040.514948,0.076301,-90.754394,140.459907,-29.109965,...,5.789265,-8.905224,-1.083720,-9.218359,2.455805,-7.726901,-1.815724,-3.433434,-2.226821,blues
3,blues.00003.wav,0.404779,0.141191,1070.119953,1596.333948,2185.028454,0.033309,-199.431144,150.099218,5.647594,...,6.087676,-2.476420,-1.073890,-2.874777,0.780976,-3.316932,0.637981,-0.619690,-3.408233,blues
4,blues.00004.wav,0.308590,0.091563,1835.494603,1748.362448,3580.945013,0.101500,-160.266031,126.198800,-35.605448,...,-2.806385,-6.934122,-7.558619,-9.173552,-4.512166,-5.453538,-0.924162,-4.409333,-11.703781,blues


In [6]:
data.shape

(1000, 28)

In [7]:
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)

In [8]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

In [9]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [11]:
len(y_train)

800

In [12]:
len(y_test)

200

In [13]:
X_train[20]

array([ 0.09004013, -0.6143136 , -0.62558909, -0.4114107 , -0.51006495,
       -0.71091224, -0.47670986,  0.54161422, -0.09331443, -0.38362656,
        0.0403447 ,  0.5734079 , -0.68161322,  0.81368615, -0.7349021 ,
        0.84803436, -0.20026021,  0.05368322,  0.32168753,  0.2664206 ,
        0.46144931, -0.77293627, -0.56070265, -0.49565438, -0.85493686,
       -0.38078167])

In [14]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))


In [15]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)

Epoch 1/20
800/800 [==============================] - 1s 1ms/step - loss: 2.1371 - acc: 0.2587
Epoch 2/20
800/800 [==============================] - 0s 46us/step - loss: 1.8455 - acc: 0.3300
Epoch 3/20
800/800 [==============================] - 0s 42us/step - loss: 1.6416 - acc: 0.4250
Epoch 4/20
800/800 [==============================] - 0s 44us/step - loss: 1.4632 - acc: 0.5262
Epoch 5/20
800/800 [==============================] - 0s 50us/step - loss: 1.3166 - acc: 0.5513
Epoch 6/20
800/800 [==============================] - 0s 51us/step - loss: 1.2008 - acc: 0.6038
Epoch 7/20
800/800 [==============================] - 0s 47us/step - loss: 1.1091 - acc: 0.6187
Epoch 8/20
800/800 [==============================] - 0s 50us/step - loss: 1.0351 - acc: 0.6638
Epoch 9/20
800/800 [==============================] - 0s 42us/step - loss: 0.9807 - acc: 0.6812
Epoch 10/20
800/800 [==============================] - 0s 45us/step - loss: 0.9214 - acc: 0.6962
Epoch 11/20
800/800 [===================

In [17]:
# Epoch 1/20
# 800/800 [==============================] - 1s 1ms/step - loss: 2.1637 - acc: 0.2250
# Epoch 2/20
# 800/800 [==============================] - 0s 64us/step - loss: 1.8532 - acc: 0.3625
# Epoch 3/20
# 800/800 [==============================] - 0s 57us/step - loss: 1.6190 - acc: 0.4062
# Epoch 4/20
# 800/800 [==============================] - 0s 56us/step - loss: 1.4568 - acc: 0.5000
# Epoch 5/20
# 800/800 [==============================] - 0s 64us/step - loss: 1.3145 - acc: 0.5337
# Epoch 6/20
# 800/800 [==============================] - 0s 60us/step - loss: 1.1780 - acc: 0.6013
# Epoch 7/20
# 800/800 [==============================] - 0s 75us/step - loss: 1.0935 - acc: 0.6650
# Epoch 8/20
# 800/800 [==============================] - 0s 61us/step - loss: 1.0032 - acc: 0.6737
# Epoch 9/20
# 800/800 [==============================] - 0s 57us/step - loss: 0.9374 - acc: 0.7000
# Epoch 10/20
# 800/800 [==============================] - 0s 87us/step - loss: 0.8768 - acc: 0.7263
# Epoch 11/20
# 800/800 [==============================] - 0s 65us/step - loss: 0.8361 - acc: 0.7375
# Epoch 12/20
# 800/800 [==============================] - 0s 110us/step - loss: 0.7891 - acc: 0.7437
# Epoch 13/20
# 800/800 [==============================] - 0s 61us/step - loss: 0.7556 - acc: 0.7563
# Epoch 14/20
# 800/800 [==============================] - 0s 74us/step - loss: 0.7123 - acc: 0.7787
# Epoch 15/20
# 800/800 [==============================] - 0s 61us/step - loss: 0.6776 - acc: 0.7900
# Epoch 16/20
# 800/800 [==============================] - 0s 66us/step - loss: 0.6432 - acc: 0.7963
# Epoch 17/20
# 800/800 [==============================] - 0s 61us/step - loss: 0.6190 - acc: 0.8100
# Epoch 18/20
# 800/800 [==============================] - 0s 64us/step - loss: 0.5978 - acc: 0.8188
# Epoch 19/20
# 800/800 [==============================] - 0s 60us/step - loss: 0.5537 - acc: 0.8337
# Epoch 20/20
# 800/800 [==============================] - 0s 64us/step - loss: 0.5374 - acc: 0.8463

In [18]:
#Link - https://github.com/priya-dwivedi/Music_Genre_Classification/blob/master/baseline_model_fma.ipynb
# Build Models

In [19]:
# from sklearn.metrics import f1_score
# from time import time

In [20]:
# def train_classifier(clf, X_train, y_train):
#     ''' Fits a classifier to the training data. '''
    
#     # Start the clock, train the classifier, then stop the clock
#     start = time()
#     clf.fit(X_train, y_train)
#     end = time()
    
#     # Print the results
#     print("Trained model in {:.4f} seconds".format(end - start))
    
# #this may not used at all    
# def predict_labels(clf, features, target):
#     ''' Makes predictions using a fit classifier based on F1 score. '''
#     #################################################F1 score astesee predict_labels(...) theke 
    
#     # Start the clock, make predictions, then stop the clock
#     start = time()
#   #  y_pred = clf.predict(features) ****Features is not defind for my dataset. if i can solve this line then
# #******************************************************{NameError: name 'y_pred' is not defined} will be 
# #######################################################solved

# ########################## ORIGIN CODE FIRST OCCURANCE OF FEATURES******************************

# #####################filepath = 'features.csv'
# ################features = pd.read_csv(filepath, index_col=0,header=[0, 1, 2], skip_blank_lines=True )

# #--------------------------------------------------------------------------------------------------

# # features is pandas read_csv dictionary 
#     end = time()
    
#     # Print and return results
#     print("Made predictions in {:.4f} seconds.".format(end - start))
#     return f1_score(target, y_pred, average='micro', pos_label = 1)



# def train_predict(clf, X_train, y_train, X_test, y_test):
#     ''' Train and predict using a classifer based on F1 score. '''
    
#     # Indicate the classifier and the training set size
#     print("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
 
#     # Train the classifier
#     train_classifier(clf, X_train, y_train)
#        #untill now no problem
        
#         #### if i switched places between the train_predict and predict_labels
    
#     # Print the results of prediction for both training and testing
#     #may not work.....comment out this below section
#     print("F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train)))
#     print("F1 score for val set: {:.4f}.".format(predict_labels(clf, X_val, y_val)))
#     print("F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test)))

In [21]:
# from pyAudioAnalysis import audioBasicIO
# from pyAudioAnalysis import ShortTermFeatures


# for g in genres:
#     pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)     
#     for filename in os.listdir(f'./MIR/genres/{g}'):
#         songname = f'./MIR/genres/{g}/{filename}'
#         y, sr = librosa.load(songname, mono=True, duration=5)
#         plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
#         plt.axis('off');
#         plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
#         plt.clf()

# [Fs, x] = audioBasicIO.read_audio_file("sample.wav")
# F, f_names = ShortTermFeatures.feature_extraction(x, Fs, 0.050*Fs, 0.025*Fs)



In [22]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.svm import SVC
# #from sklearn.naive_bayes import GaussianNB
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier

# # TODO: Initialize the three models
# clf_A = DecisionTreeClassifier(random_state=10, max_depth =4)
# clf_B = SVC()
# clf_C = LogisticRegression()
# clf_D = RandomForestClassifier(random_state=10, max_depth=30, n_estimators=300, min_samples_leaf=6, min_impurity_decrease=0.0002,
#                      class_weight='balanced')

# for clf in [clf_A, clf_B, clf_C, clf_D]:
#     print("\n{}: \n".format(clf.__class__.__name__))
#     train_predict(clf, X_train, y_train, X_test, y_test)   # X_val turned to X_test, y_val to y_test



In [23]:
### Classification of Music into different Genres using Keras
####https://medium.com/@sdoshi579/classification-of-music-into-different-genres-using-keras-82ab5339efe0

In [48]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import csv# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler#Keras
import keras

In [49]:
###create a CSV file

header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [50]:
###calculate all the features.

file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    #for filename in os.listdir(f'./genres/{g}'):              ###############Eroor****FileNotFoundError: [WinError 3] The system cannot find the path specified: './genres/blues'
    for filename in os.listdir(f'./MIR/genres/{g}'):
        songname = f'./MIR/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)   ###errror****FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\SCC\\genres\\blues\\blues.00000.wav'
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rmse(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())
            
            
            
            #ran with out an error

In [51]:
data = pd.read_csv('data.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00000.wav,0.349943,0.130225,1784.420446,2002.650192,3806.485316,0.083066,-113.596742,121.557302,-19.158825,...,8.810668,-3.667367,5.751690,-5.162761,0.750947,-1.691937,-0.409954,-2.300208,1.219928,blues
1,blues.00001.wav,0.340983,0.095918,1529.835316,2038.617579,3548.820207,0.056044,-207.556796,124.006717,8.930562,...,5.376802,-2.239119,4.216963,-6.012273,0.936109,-0.716537,0.293875,-0.287431,0.531573,blues
2,blues.00002.wav,0.363603,0.175573,1552.481958,1747.165985,3040.514948,0.076301,-90.754394,140.459907,-29.109965,...,5.789265,-8.905224,-1.083720,-9.218359,2.455805,-7.726901,-1.815724,-3.433434,-2.226821,blues
3,blues.00003.wav,0.404779,0.141191,1070.119953,1596.333948,2185.028454,0.033309,-199.431144,150.099218,5.647594,...,6.087676,-2.476420,-1.073890,-2.874777,0.780976,-3.316932,0.637981,-0.619690,-3.408233,blues
4,blues.00004.wav,0.308590,0.091563,1835.494603,1748.362448,3580.945013,0.101500,-160.266031,126.198800,-35.605448,...,-2.806385,-6.934122,-7.558619,-9.173552,-4.512166,-5.453538,-0.924162,-4.409333,-11.703781,blues


In [52]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 

In [53]:
data.iloc[:, : -1]

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,blues.00000.wav,0.349943,0.130225,1784.420446,2002.650192,3806.485316,0.083066,-113.596742,121.557302,-19.158825,...,-8.324323,8.810668,-3.667367,5.751690,-5.162761,0.750947,-1.691937,-0.409954,-2.300208,1.219928
1,blues.00001.wav,0.340983,0.095918,1529.835316,2038.617579,3548.820207,0.056044,-207.556796,124.006717,8.930562,...,-5.560387,5.376802,-2.239119,4.216963,-6.012273,0.936109,-0.716537,0.293875,-0.287431,0.531573
2,blues.00002.wav,0.363603,0.175573,1552.481958,1747.165985,3040.514948,0.076301,-90.754394,140.459907,-29.109965,...,-13.123111,5.789265,-8.905224,-1.083720,-9.218359,2.455805,-7.726901,-1.815724,-3.433434,-2.226821
3,blues.00003.wav,0.404779,0.141191,1070.119953,1596.333948,2185.028454,0.033309,-199.431144,150.099218,5.647594,...,-3.196314,6.087676,-2.476420,-1.073890,-2.874777,0.780976,-3.316932,0.637981,-0.619690,-3.408233
4,blues.00004.wav,0.308590,0.091563,1835.494603,1748.362448,3580.945013,0.101500,-160.266031,126.198800,-35.605448,...,-13.083820,-2.806385,-6.934122,-7.558619,-9.173552,-4.512166,-5.453538,-0.924162,-4.409333,-11.703781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,rock.00095.wav,0.351991,0.079469,2008.581132,2106.617024,4254.215942,0.089267,-153.632304,109.857258,-23.085707,...,-12.380283,12.350816,-12.306271,5.780973,-10.279924,1.791489,-13.304210,2.473193,-6.717573,-1.189238
996,rock.00096.wav,0.398653,0.076452,2006.051164,2068.327905,4147.374921,0.097659,-142.424217,116.219780,-32.177077,...,-12.982495,14.065346,-16.394693,4.152589,-4.350760,3.736455,-10.845639,1.875218,-7.459579,-2.802677
997,rock.00097.wav,0.432103,0.081617,2077.190361,1926.989678,4030.767293,0.121824,-125.031313,115.194965,-47.993506,...,-16.048916,10.481961,-17.951917,1.211113,-11.534863,1.844773,-12.847902,3.447424,-12.594178,-2.107003
998,rock.00098.wav,0.362349,0.083888,1398.672358,1818.148469,3014.740104,0.048731,-224.972299,123.656889,-9.754534,...,-9.000729,7.683082,-10.071786,-0.255097,-5.276486,-2.816289,-4.416438,1.558265,-5.043121,-3.585596


In [54]:
#######np.array(data.iloc[:, : -1] , dtype = float) ##########here's the error

####np.array(data.iloc[:, : -1] , dtype.astype = float)

np.array(data.iloc[:, : -1])




array([['blues.00000.wav', 0.34994319703898874, 0.13022463023662567, ...,
        -0.4099536931866388, -2.3002080931099553, 1.219928131251096],
       ['blues.00001.wav', 0.340983161628006, 0.09591842442750932, ...,
        0.29387517328195234, -0.2874306648121582, 0.5315729228628573],
       ['blues.00002.wav', 0.36360283849610303, 0.17557303607463834, ...,
        -1.815723851127716, -3.4334342717656465, -2.2268214410307183],
       ...,
       ['rock.00097.wav', 0.4321034468964737, 0.08161668479442596, ...,
        3.447423910030597, -12.594177514097485, -2.10700255491672],
       ['rock.00098.wav', 0.3623490158126501, 0.08388779312372208, ...,
        1.5582648453800767, -5.043121274989656, -3.5855956471307313],
       ['rock.00099.wav', 0.35819512534874665, 0.05446073040366173, ...,
        0.023642584756170492, -2.0220346710460277, 1.158525253018968]],
      dtype=object)

In [58]:
scaler = StandardScaler()


#convertion
x = np.array(data.iloc[:, :-1])
Y = x.astype(np.float)


X = scaler.fit_transform(Y)

ValueError: could not convert string to float: 'rock.00099.wav'

In [62]:
# convert X into dataframe
X_pd = pd.DataFrame(data=x)
# replace all instances of URC with 0 
X_replace = for item in X_pd:
                
                X_pd.replace(f'rock.000{num}.wav',0, regex=True)

# # convert it back to numpy array
X_np = X_replace.values
# # set the object type as float
X_fa = X_np.astype(dtype = float)

ValueError: could not convert string to float: 'rock.00099.wav'

In [35]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

ValueError: could not convert string to float: 'blues.00000.wav'